In [1]:
import sys
sys.path.append("../")
from data_loaders import load_dataset, load_inference_data, promptTechList, modelList,load_data_all

# Match Answer

In [2]:
dataset, inference_data = load_data_all(problem_name='IndirectBooleanComputation')

In [119]:
print(inference_data['wizardmath-13b']['CoT'][99]['prompt'])

Compute the value of !(((a && a) || (!a && !a)) && a) || a, where a=True. Let's think step by step carefully.


In [114]:
print(inference_data['wizardmath-13b']['CoT'][99]['response'])



Step 1: Evaluate the innermost operands (a and !a)
Since a is True, we have:
a = True
!a = False

Step 2: Evaluate the middle operand (a || !a)
Using the values from Step 1, we have:
a || !a = True || False = False

Step 3: Evaluate the outermost operand ((a || !a) && a)
Using the values from Step 2, we have:
((a || !a) && a) = (False && True) = False

Step 4: Evaluate the entire expression
Using the values from Steps 1-3, we have:
!(((a && a) || (!a && !a)) && a) || a = !(False) || True = True

So, the final answer is True.


In [12]:
selected_lis = [index for index in range(len(dataset)) if len(dataset[index]['Assignment']) == 2 and dataset[index]['Complexity'] <= 10] 

In [16]:
selected_lis[1]

237

In [21]:
index = 241
print(inference_data['llama2-13b']['CoT'][index]['prompt'])
print("---------------------------------------------------------------------------")
print(inference_data['llama2-13b']['CoT'][index]['response'])
print("---------------------------------------------------------------------------")
print(inference_data['wizardmath-13b']['CoT'][index]['response'])

Compute the value of !(a || (a && b)), where a=False,b=True. Let's think step by step carefully.
---------------------------------------------------------------------------


Firstly, we need to compute the value of a || b, which is equal to the value of a || True.

Now, we know that a is False, so the only possible values that can be obtained from a || True are:

1. False || True = False
2. True || True = True

Therefore, the value of a || (a && b) is equal to False || True = False.

So, the final result is !(a || (a && b)) = !False = True.
---------------------------------------------------------------------------


Step 1: Understand the given values of a and b.
We are given that a = False and b = True.

Step 2: Evaluate the first part of the expression, a || (a && b).
Since a is False, the entire expression inside the parentheses (a && b) will be considered as False, because in an AND operation, both operands must be True for the result to be True. So, we have:
a || (a && b) = Fals

## SAT

In [3]:
SAT_Dataset = load_dataset('SAT')

SAT_inference_data = load_inference_data(
    'SAT', 'CoT', 'llama2-13b')

In [4]:
import re


def matchAnswer(res, SAT_data):
    SAT_re = '|'.join(
        [
            '.{2,5}'.join(item) for item in [
                [
                    var + ' = ' + str(val) for var, val in zip(SAT_data['Variables'], combination)
                ] for combination in SAT_data['SAT']
            ]
        ] + [
            '\(' + '\W{2,5}'.join(item) + '\)' for item in [
                [str(item) for item in SAT_item]
                for SAT_item in SAT_data['SAT']
            ]
        ]
    )
    # return SAT_re
    matchCase = re.search(SAT_re, res, re.I)

    return matchCase

In [ ]:
cases = [matchAnswer(response['response'], dataObject)
         for dataObject, response in zip(SAT_Dataset, SAT_inference_data)]

In [43]:
m = [(x,y) for x,y in zip(cases,SAT_inference_data) if x != None]

In [55]:
print(m[5][1]['prompt'])

Give a choice of boolean variables (a, b) to make the logic expression "(!a && b && a) || b" yield True. Let's think step by step carefully.


In [59]:
print(m[9][1]['response'])



Hint: You don't need to worry about short circuits or precedence, just focus on finding a way to make it true.

Solution:

Let a = true and b = false.

Then, a && a is true, and (!a || a || b) is true because !a is false and a is true, and b is false.

So, the expression a && a && (!a || a || b) yields true when a = true and b = false.


## TautologyQ

In [ ]:
from TautologyQ import response_evaluator
TautologyQ_Dataset = load_dataset('TautologyQ')

TautologyQ_inference_data = load_inference_data(
    'TautologyQ', 'Few-shot-CoT', 'llama2-13b')

In [ ]:
import re

def matchAnswer(res, ans):

    lower_res = res.lower()
    if ans == False:
        r = "(\W no\W)|(not a tautology)"
    else:
        r = "(\W yes\W)|(is a tautology)"

    case = re.search(r, lower_res)
    return case

In [ ]:
cases = [matchAnswer(response['response'], dataObject['TautologyQ'])
         for dataObject, response in zip(TautologyQ_Dataset, TautologyQ_inference_data)]

## SATCount

In [ ]:
from SATCount import response_evaluator
SATCount_Dataset = load_dataset('SATCount')

SATCount_inference_data = load_inference_data('SATCount', 'CoT', 'llama2-13b')

In [ ]:
import re


def matchAnswer(res, ans):

    lower_res = res.lower()

    num_to_Eng = {
        0: 'no', 1: 'one', 2: 'two', 3: 'three',
        4: 'four', 5: 'five', 6: 'six', 7: 'seven',
        8: 'eight', 9: 'nine', 10: 'ten', 11: 'eleven',
        12: 'twelve', 13: 'thirteen', 14: 'fourteen', 15: 'fifteen'
    }

    r = f"(((is)|(are))\s(only\s)?(({ans})|({num_to_Eng[ans]}))\s(possible\s)?(combination)(s?))|(a: (({ans})|({num_to_Eng[ans]})))"
    case = re.search(r, lower_res)

    return case

In [ ]:
result = [response_evaluator(response['response'], dataObject['Answer'])
          for dataObject, response in zip(SATCount_Dataset, SATCount_inference_data)]

In [ ]:
accuracy(result)

0.10101101281819823

In [ ]:
cases = [matchAnswer(response['response'], dataObject['Answer'])
         for dataObject, response in zip(SATCount_Dataset, SATCount_inference_data)]

In [ ]:
[case for case in cases if case != None]